In [1]:
import torch
import random
import datetime
import pandas as pd
import numpy as np
import os

from torch.utils.data import Dataset
from src.datasets import RL4RS, ContentWise, DummyData
from src.utils import train, get_dummy_data, get_train_val_test_tmatrix_tnumitems, fit_treshold
from src.embeddings import RecsysEmbedding
from sklearn.linear_model import LogisticRegression

experiment_name = 'NeuralClickModelDiffSample'
device = 'cuda:0'
seed = 7331
pkl_path = '../pkl/'


random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

In [2]:
def flatten(true, pred, mask, to_cpu=True):
    mask = mask.flatten()
    nnz_idx = mask.nonzero()[:, 0]
    true, pred = [x.flatten()[nnz_idx] for x in [true, pred]]
    if to_cpu:
        true, pred = [x.cpu().numpy() for x in [true, pred]]
    return true, pred

# Модель

In [3]:
import torch.nn as nn
import torch.nn.functional as F

class SlatewiseGRU(nn.Module):
    def __init__(self, embedding, readout=False):
        super().__init__()
        
        self.embedding = embedding
        
        self.emb_dim = embedding.embedding_dim
        self.rnn_layer = nn.GRU(
            input_size=self.emb_dim * 2, 
            hidden_size=self.emb_dim, 
            batch_first=True
        )
        self.out_layer = nn.Linear(self.emb_dim, 1)
        
        self.thr = -1.5
        self.readout = readout
        self.readout_mode = 'diff_sample' # ['soft' ,'threshold', 'sample', 'diff_sample']
        
        self.calibration = False
        self.w = 1
        self.b = 0

    def forward(self, batch):
        item_embs, user_embs = self.embedding(batch)
        shp = item_embs.shape
        max_sequence = item_embs.size(1)
        # ilya format:
        # 'items': (batch, slate, 2*embedding_dim ) 2, нужно для readout, по умолчанию ноли на половине эмбеддинга
        # 'clicks': (batch, slate)
        # 'users': (1, batch, embedding_dim), 
        # 'mask': (batch, slate)
        
        x = {}
        x['items'] = torch.cat(
            [
                item_embs.flatten(0,1),
                torch.zeros_like(item_embs.flatten(0,1)),
            ],
            dim = -1
        )
                
        if self.training:
            indices = (batch['length'] - 1)
        else:
            indices = (batch['in_length'] - 1)
        indices[indices<0] = 0
        indices = indices[:, None, None].repeat(1, 1, user_embs.size(-1))
        user_embs = user_embs.gather(1, indices).squeeze(-2).unsqueeze(0)
        x['users'] = user_embs.repeat_interleave(max_sequence, 1)
        x['clicks'] = (batch['responses'].flatten(0,1) > 0 ).int().clone()
        x['mask'] = batch['slates_mask'].flatten(0,1).clone()
        
        items = x['items']
        h = x['users']
        
        if self.readout:
            res = []
            seq_len = items.shape[1]
            for i in range(seq_len):
#                 print(items[:,[i],:])
                output, h = self.rnn_layer(items[:,[i],:], h)
                y = self.out_layer(output)[:, :, 0]
                
                # readout
                if i + 1 < seq_len:
                    if self.readout_mode == 'threshold':
                        items[:, [i+1], self.emb_dim:] *= (y.detach()[:, :, None] > self.thr).to(torch.float32)
                    elif self.readout_mode == 'soft':
                        items[:, [i+1], self.emb_dim:] *= torch.sigmoid(y)[:, :, None]
                    elif self.readout_mode == 'diff_sample' or self.readout_mode == 'sample':
                        eps = 1e-8
                        gumbel_sample = -( (torch.rand_like(y) + eps).log() / (torch.rand_like(y) + eps).log() + eps).log()
                        T = 0.3
                        bernoulli_sample = torch.sigmoid( (nn.LogSigmoid()(self.w * y + self.b) + gumbel_sample) / T )
                        if self.readout_mode == 'sample':
                            bernoulli_sample = bernoulli_sample.detach()
                        items[:, [i+1], self.emb_dim:] *= bernoulli_sample[:, :, None]
                    else:
                        raise
                    
                res.append(y)
        
            y = torch.cat(res, axis=1)
            
        else:
            items[:, 1:, self.emb_dim:] *= x['clicks'][:, :-1, None]
            rnn_out, _ = self.rnn_layer(items, h)
            y = self.out_layer(rnn_out)[:, :, 0]
        
        
        if self.calibration and self.training:
            clicks_flat, logits_flat = flatten(x['clicks'], y.detach(), x['mask'])
            logreg = LogisticRegression()
            logreg.fit(logits_flat[:, None], clicks_flat)
            γ = 0.3
            self.w = (1 - γ) * self.w + γ * logreg.coef_[0, 0]
            self.b = (1 - γ) * self.b + γ * logreg.intercept_[0]
            y = self.w * y + self.b
        else:
            y = self.w * y + self.b
            
        return y.reshape(shp[:-1])

# Игрушечный датасет: проверим, что сходится к идеальным метрикам

In [4]:
d = DummyData()
dummy_loader, dummy_matrix = get_dummy_data(d)

model = SlatewiseGRU(
    RecsysEmbedding(d.n_items, dummy_matrix, embeddings='svd', embedding_dim = 2),
    readout=False
).to('cpu')

train(
    model, 
    dummy_loader, dummy_loader, dummy_loader,
    device=device, lr=1e-3, num_epochs=5000, dummy=True,
    silent=True,
)


biulding affinity matrix...


3it [00:00, 3492.34it/s]


Test before learning: {'f1': 0.0, 'roc-auc': 0.3333333134651184, 'accuracy': 0.75}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.3333333134651184 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.3333333134651184 | 
Val update: epoch: 41 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 0.6666666269302368 | 
Val update: epoch: 44 |accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | treshold: 0.01
Test: accuracy: 0.25 | f1: 0.4000000059604645 | auc: 1.0 | 
Val update: epoch: 84 |accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | treshold: 0.36000000000000004
Test: accuracy: 0.75 | f1: 0.6666666865348816 | auc: 1.0 | 
Val update: epoch: 117 |accuracy: 1.0 | f1: 1.0 | auc: 1.0 | treshold: 0.38
Test: accuracy: 1.0 | f1: 1.0 | auc: 1.0 | 


(SlatewiseGRU(
   (embedding): RecsysEmbedding()
   (rnn_layer): GRU(4, 2, batch_first=True)
   (out_layer): Linear(in_features=2, out_features=1, bias=True)
 ),
 {'f1': 1.0, 'roc-auc': 1.0, 'accuracy': 1.0})

# ContentWise

In [5]:
content_wise_results = []
dataset = ContentWise.load(os.path.join(pkl_path, 'cw.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=150)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

20216 data points among 108 batches


In [6]:
for embeddings in ['svd', 'neural']:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")
    model = SlatewiseGRU(
        RecsysEmbedding(train_num_items, train_user_item_matrix, embeddings=embeddings).to('cpu'),
    ).to(device)

    _, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
       silent=True, 
    )
    
    metrics['embeddings'] = embeddings
    content_wise_results.append(metrics)


Evaluating NeuralClickModelDiffSample with svd embeddings
Test before learning: {'f1': 0.17426511645317078, 'roc-auc': 0.4757869243621826, 'accuracy': 0.11266712099313736}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.4177514910697937 | f1: 0.20474010705947876 | auc: 0.6037461757659912 | treshold: 0.09
Test: accuracy: 0.4186418950557709 | f1: 0.2041238397359848 | auc: 0.6021729111671448 | 
Val update: epoch: 2 |accuracy: 0.7755369544029236 | f1: 0.2239958792924881 | auc: 0.6198201775550842 | treshold: 0.12
Test: accuracy: 0.7755268812179565 | f1: 0.21790620684623718 | auc: 0.6188664436340332 | 
Val update: epoch: 3 |accuracy: 0.8025559186935425 | f1: 0.2338458001613617 | auc: 0.634839653968811 | treshold: 0.12
Test: accuracy: 0.8028401136398315 | f1: 0.23125405609607697 | auc: 0.6313900351524353 | 
Val update: epoch: 4 |accuracy: 0.8038070797920227 | f1: 0.2359628826379776 | auc: 0.6448588967323303 | treshold: 0.13
Test: accuracy: 0.8047133684158325 | f1: 0.23422783613204956 | auc: 0.6403693556785583 | 
Val update: epoch: 5 |accuracy: 0.8056540489196777 | f1: 0.2453441321849823 | auc: 0.6500104069709778 | treshold: 0.13
Test: accuracy: 0.8058916926383972 | f1: 0.241

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.29635974764823914 | f1: 0.19517181813716888 | auc: 0.611788809299469 | treshold: 0.11
Test: accuracy: 0.30129164457321167 | f1: 0.19452813267707825 | auc: 0.6132309436798096 | 
Val update: epoch: 1 |accuracy: 0.7892847657203674 | f1: 0.24205732345581055 | auc: 0.6545871496200562 | treshold: 0.14
Test: accuracy: 0.7893194556236267 | f1: 0.23966851830482483 | auc: 0.6565821170806885 | 
Val update: epoch: 2 |accuracy: 0.8549703359603882 | f1: 0.2453693002462387 | auc: 0.6861916184425354 | treshold: 0.14
Test: accuracy: 0.8545660376548767 | f1: 0.24405182898044586 | auc: 0.6843408346176147 | 
Val update: epoch: 3 |accuracy: 0.8784592747688293 | f1: 0.2459804117679596 | auc: 0.7051868438720703 | treshold: 0.15000000000000002
Test: accuracy: 0.8789334297180176 | f1: 0.246662899851799 | auc: 0.7059399485588074 | 
Val update: epoch: 4 |accuracy: 0.8829277157783508 | f1: 0.2607223391532898 | auc: 0.7141774892807007 | treshold: 0.16
Test: accuracy: 0.88396406173

In [7]:
pd.DataFrame(content_wise_results).to_csv(f'results/cw_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

# RL4RS

In [8]:
rl4rs_results = []
dataset = RL4RS.load(os.path.join(pkl_path, 'rl4rs.pkl'))
(
    train_loader, 
    val_loader, 
    test_loader, 
    train_user_item_matrix, 
    train_num_items 
) = get_train_val_test_tmatrix_tnumitems(dataset, batch_size=350)

print(f"{len(dataset)} data points among {len(train_loader)} batches")

45942 data points among 106 batches


In [9]:
for embeddings in ['explicit', 'neural', 'svd',  ]:
    print(f"\nEvaluating {experiment_name} with {embeddings} embeddings")

    model = SlatewiseGRU(
        RecsysEmbedding(
            train_num_items, 
            train_user_item_matrix, 
            embeddings=embeddings,
            embedding_dim=40
        ),
    ).to(device)

    best_model, metrics = train(
        model, 
        train_loader, val_loader, test_loader, 
        device=device, lr=1e-3, num_epochs=5000, early_stopping=7,
        silent=True
    )
    
    metrics['embeddings'] = embeddings
    rl4rs_results.append(metrics)


Evaluating NeuralClickModelDiffSample with explicit embeddings
Test before learning: {'f1': 0.37980231642723083, 'roc-auc': 0.42735081911087036, 'accuracy': 0.3976786434650421}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7717554569244385 | f1: 0.8202852606773376 | auc: 0.8371963500976562 | treshold: 0.43
Test: accuracy: 0.773280143737793 | f1: 0.8214435577392578 | auc: 0.8381221294403076 | 
Val update: epoch: 1 |accuracy: 0.7920717597007751 | f1: 0.8535808324813843 | auc: 0.8681284189224243 | treshold: 0.37
Test: accuracy: 0.7912707328796387 | f1: 0.8526962399482727 | auc: 0.8670390248298645 | 
Val update: epoch: 2 |accuracy: 0.7967880964279175 | f1: 0.8574289083480835 | auc: 0.8883963823318481 | treshold: 0.36000000000000004
Test: accuracy: 0.7965179681777954 | f1: 0.8569437265396118 | auc: 0.8881847858428955 | 
Val update: epoch: 3 |accuracy: 0.8254728317260742 | f1: 0.8622006773948669 | auc: 0.8992475271224976 | treshold: 0.41000000000000003
Test: accuracy: 0.8270100355148315 | f1: 0.8632749915122986 | auc: 0.8994544744491577 | 
Val update: epoch: 5 |accuracy: 0.8422580361366272 | f1: 0.8798717856407166 | auc: 0.9118956923484802 | treshold: 0.37
Test: accuracy: 0.84

train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7641609907150269 | f1: 0.8134172558784485 | auc: 0.8337212800979614 | treshold: 0.4
Test: accuracy: 0.7661951184272766 | f1: 0.8146316409111023 | auc: 0.8369123339653015 | 
Val update: epoch: 1 |accuracy: 0.8060029745101929 | f1: 0.8488457798957825 | auc: 0.8800385594367981 | treshold: 0.39
Test: accuracy: 0.8103010654449463 | f1: 0.8518441319465637 | auc: 0.8818862438201904 | 
Val update: epoch: 2 |accuracy: 0.8223528265953064 | f1: 0.8693455457687378 | auc: 0.9029510021209717 | treshold: 0.37
Test: accuracy: 0.8232136368751526 | f1: 0.8698716759681702 | auc: 0.9033089280128479 | 
Val update: epoch: 4 |accuracy: 0.8277705311775208 | f1: 0.8626111745834351 | auc: 0.9041354656219482 | treshold: 0.43
Test: accuracy: 0.8303953409194946 | f1: 0.865027129650116 | auc: 0.9043761491775513 | 
Val update: epoch: 5 |accuracy: 0.8415808081626892 | f1: 0.8788158893585205 | auc: 0.9145747423171997 | treshold: 0.36000000000000004
Test: accuracy: 0.8424132466316223 |

/opt/conda/envs/myenv/lib/python3.10/site-packages/torchmetrics/utilities/prints.py:43: UserWarning: No negative samples in targets, false positive value should be meaningless. Returning zero tensor in false positive score
  warnings.warn(*args, **kwargs)  # noqa: B028



Evaluating NeuralClickModelDiffSample with svd embeddings
Test before learning: {'f1': 0.6839137077331543, 'roc-auc': 0.43305230140686035, 'accuracy': 0.5364526510238647}


train:   0%|          | 0/5000 [00:00<?, ?it/s]

Val update: epoch: 0 |accuracy: 0.7100808024406433 | f1: 0.8049276471138 | auc: 0.7366717457771301 | treshold: 0.22
Test: accuracy: 0.7096360921859741 | f1: 0.8042450547218323 | auc: 0.7383829951286316 | 
Val update: epoch: 1 |accuracy: 0.7064528465270996 | f1: 0.8111531138420105 | auc: 0.7686601877212524 | treshold: 0.31
Test: accuracy: 0.7047757506370544 | f1: 0.8098078966140747 | auc: 0.7712188363075256 | 
Val update: epoch: 2 |accuracy: 0.7042277455329895 | f1: 0.8114641904830933 | auc: 0.7883633375167847 | treshold: 0.36000000000000004
Test: accuracy: 0.7022609114646912 | f1: 0.8098701238632202 | auc: 0.790867805480957 | 
Val update: epoch: 3 |accuracy: 0.7081700563430786 | f1: 0.8136410117149353 | auc: 0.7976177334785461 | treshold: 0.4
Test: accuracy: 0.7073147296905518 | f1: 0.8125735521316528 | auc: 0.8012973666191101 | 
Val update: epoch: 4 |accuracy: 0.7656121253967285 | f1: 0.8246004581451416 | auc: 0.8103487491607666 | treshold: 0.38
Test: accuracy: 0.7687099575996399 | f1

In [10]:
pd.DataFrame(rl4rs_results).to_csv(f'results/rl4rs_{experiment_name}.csv')
del dataset, train_loader, val_loader, test_loader, train_user_item_matrix, train_num_items

In [11]:
rl4rs_results

[{'f1': 0.8818536996841431,
  'roc-auc': 0.9157841205596924,
  'accuracy': 0.84785395860672,
  'embeddings': 'explicit'},
 {'f1': 0.8836547136306763,
  'roc-auc': 0.9185827970504761,
  'accuracy': 0.8478297591209412,
  'embeddings': 'neural'},
 {'f1': 0.8675724267959595,
  'roc-auc': 0.908454179763794,
  'accuracy': 0.8352557420730591,
  'embeddings': 'svd'}]